In [1]:
from langchain_community.vectorstores import Chroma

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

/mnt/c/Users/iMaji/Documents/AI Eng/TJ-Bot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1759014756.613488    2732 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [175]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("doc/web_data.md", mode="single")
docs = loader.load()

In [176]:
len(docs[0].page_content)

23075

In [137]:
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [177]:
def split_text(documents: list[Document]):
  """
  Split the text content of the given list of Document objects into smaller chunks.
  Args:
    documents (list[Document]): List of Document objects containing text content to split.
  Returns:
    list[Document]: List of Document objects representing the split text chunks.
  """
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # Size of each chunk in characters
    chunk_overlap=50, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
    separators=["\n"], # List of separators to use for splitting
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

In [178]:
chunk = split_text(docs) # Call the split_text function with the loaded documents

Split 1 documents into 113 chunks.
Layanan

Angkutan Pengumpan

Layanan angkutan umum yang melayani titik perhentian di luar jaringan jalur BRT Transjakarta.

BRT
{'source': 'doc/web_data.md', 'start_index': 0}


In [179]:
chunk[0]

Document(metadata={'source': 'doc/web_data.md', 'start_index': 0}, page_content='Layanan\n\nAngkutan Pengumpan\n\nLayanan angkutan umum yang melayani titik perhentian di luar jaringan jalur BRT Transjakarta.\n\nBRT')

In [180]:
chunk[1]

Document(metadata={'source': 'doc/web_data.md', 'start_index': 124}, page_content='BRT\n\nLayanan angkutan umum yang melayani titik perhentian di jalur khusus BRT (busway) Transjakarta.\n\nMikrotrans')

In [152]:
from langchain_community.vectorstores.utils import filter_complex_metadata

docs_clean = filter_complex_metadata(chunk)

In [187]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = Chroma(
    collection_name="bus",
    embedding_function=embedding,
    persist_directory="./chroma_b"  # simpan di sini
)


In [188]:
vector_store.add_documents(chunk)

['9cc82cfe-4d79-4742-b865-843f4e1489d7',
 'bc228abc-9009-4597-b95a-c6a15deaf632',
 'cc4afd2a-c627-4ee6-b914-6fa81094b4f2',
 '86288727-2e5e-4900-8485-82519b92e3d0',
 'd591eed5-6193-443d-b074-4b91ff429c6f',
 'b181a097-e249-4d6e-85d0-34c0e96ad232',
 '66b6c5ce-7b92-4e1a-b7e7-1636dfd98d11',
 '3f0f33ed-7cd6-4826-b851-816ee6dbc552',
 'aa8bcae2-94c6-449a-8242-fe33e9fab9eb',
 'f0630372-c7b0-4425-877d-8a09f59b63b9',
 '89dd1d1a-f59b-490b-912a-12018b21076b',
 'ff3fb396-cb11-4679-b93d-54b8f8918442',
 '2c687766-5b1e-4885-8285-bc4386ddb64b',
 '2e6675ee-0e4f-485c-91c5-a3e592656fe5',
 '714ada13-abef-4586-9e8c-d8a72f144ae7',
 'bd809335-fb05-4830-865e-3b2664259fdd',
 'defd000e-2110-4849-b2f3-01b89138431a',
 'cf07af48-55df-4796-9efe-4e506855daf8',
 'afb0d70c-a497-407b-b6c6-cb53431ff81b',
 '095c7bfb-22bb-463d-816d-be203bc9fd16',
 'a55c19b1-1dc8-44ee-9c83-a5012fad0e7d',
 '20dea903-94e9-4a32-b809-39e176234471',
 '1db4acee-1ecf-4cdc-8164-c0d1872d0277',
 '6a70b433-a5f6-4ac2-a96a-e1d371e2cbbd',
 '51fc8a7c-68a9-

In [205]:
q = "BRT layanan"  # Example query
results = vector_store.similarity_search_with_relevance_scores(q, k=5)
results



[(Document(metadata={'start_index': 17706, 'source': 'doc/web_data.md'}, page_content='Capaian : Kegiatan ini sebagai bentuk kepedulian Transjakarta untuk memberikan sanitasi layak di 10 titik di wilayah Kebon Pala, kecamatan Makassar.'),
  0.33425634667628645),
 (Document(metadata={'start_index': 0, 'source': 'doc/web_data.md'}, page_content='Layanan\n\nAngkutan Pengumpan\n\nLayanan angkutan umum yang melayani titik perhentian di luar jaringan jalur BRT Transjakarta.\n\nBRT'),
  0.2998986572386908),
 (Document(metadata={'source': 'doc/web_data.md', 'start_index': 124}, page_content='BRT\n\nLayanan angkutan umum yang melayani titik perhentian di jalur khusus BRT (busway) Transjakarta.\n\nMikrotrans'),
  0.26591021634089906),
 (Document(metadata={'source': 'doc/web_data.md', 'start_index': 11292}, page_content='Kebijakan & Pedoman terkait lainnya.\n\nPrinsip-prinsip GCG'),
  0.1780073160937291),
 (Document(metadata={'start_index': 22907, 'source': 'doc/web_data.md'}, page_content='Selai

In [199]:
q = " visi misi"
result_2 = vector_store.search(q, k=3, search_type="similarity")
result_2

[Document(metadata={'start_index': 8607, 'source': 'doc/web_data.md'}, page_content='Visi Misi Transjakarta\n\nVisi : Menghubungkan Kehidupan Jakarta Misi : Bersama-sama menyediakan layanan transportasi terintegrasi yang memudahkan dan membahagiakan kehidupan Jakarta.'),
 Document(metadata={'start_index': 4413, 'source': 'doc/web_data.md'}, page_content='Logo Baru TransJakarta 2014'),
 Document(metadata={'start_index': 3569, 'source': 'doc/web_data.md'}, page_content='\nBadan Layanan Umum TransJakarta mengganti logo TransJakarta. Mulai 15 Juni sosialisasi penggantian logo ini dilaksanakan. Huruf T dan J yang menyatu melambangkan sistem operasional yang integrasi. Huruf T yang tak terputus juga mengandung arti layanan TransJakarta yang dapat menjangkau seluruh wilayah Jakarta. Sedangkan warna merah dalam kotak mewakili koridor pertama TransJakarta yang mengawali jaringan bus berlajur khusus pertama di Indonesia.')]